# Experiments with smaller neural net architectures

In [44]:
import os
import pickle
import socket
import struct
import time
from pathlib import Path
from typing import List, Union, Tuple
import math

import h5py
import numpy as np
import tenseal as ts
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from icecream import ic
ic.configureOutput(includeContext=True)
from tenseal.enc_context import Context
from tenseal.tensors.ckkstensor import CKKSTensor
from tenseal.tensors.ckksvector import CKKSVector
from tenseal.tensors.plaintensor import PlainTensor
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, Dataset

print(f'torch version: {torch.__version__}')
print(f'tenseal version: {ts.__version__}')

project_path = Path.cwd().parent.parent
print(f'project_path: {project_path}')

torch version: 1.8.1+cu102
tenseal version: 0.3.5
project_path: /home/dk/Desktop/split-learning-1D-HE


In [45]:
class ECG(Dataset):
    """The class used by the client to load the dataset

    Args:
        Dataset ([type]): [description]
    """
    def __init__(self, data_dir: str, train_name: str, test_name: str, train=True):
        if train:
            with h5py.File(project_path/data_dir/train_name, 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(project_path/data_dir/test_name, 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

In [46]:
batch_size = 4
train_dataset = ECG('data', 'train_ecg.hdf5', 'test_ecg.hdf5', train=True)
test_dataset = ECG('data', 'train_ecg.hdf5', 'test_ecg.hdf5', train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [47]:
x_train, y_train = next(iter(train_loader))
print(x_train.size())
print(y_train.size())
total_batch = len(train_loader)
print(total_batch)

torch.Size([4, 1, 128])
torch.Size([4])
3312


In [48]:
run = 1
epoch = 400
lr = 0.001
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(f'device: {torch.cuda.get_device_name(0)}')

device: NVIDIA GeForce GTX 1070 Ti


In [49]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [50]:
def train(nrun, model):
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    train_losses = list()
    train_accs = list()
    train_times = list()

    test_losses = list()
    test_accs = list()
    test_times = list()

    best_test_acc = 0  # best test accuracy 

    for e in range(epoch):
        train_start = time.time()
        print("Epoch {} - ".format(e+1), end='')

        # train
        train_loss = 0.0
        correct, total = 0, 0
        for i, batch in enumerate(train_loader):
            x, label = batch  # get feature and label from a batch
            x, label = x.to(device), label.to(device)  # send to device
            optimizer.zero_grad()  # init all grads to zero
            output = model(x)  # forward propagation
            loss = criterion(output, label)  # calculate loss
            loss.backward()  # backward propagation
            optimizer.step()  # weight update

            train_loss += loss.item()
            correct += torch.sum(output.argmax(dim=1) == label).item()
            total += len(label)
            if i == 9: break
        train_losses.append(train_loss / len(train_loader))
        train_accs.append(correct / total)
        train_end = time.time()
        train_times.append(train_end-train_start)
        print("loss: {:.4f}, acc: {:.2f}%".format(train_losses[-1], train_accs[-1]*100), end=' / ')
        

        # test
        test_start = time.time()
        with torch.no_grad():
            test_loss = 0.0
            correct, total = 0, 0
            for _, batch in enumerate(test_loader):
                x, label = batch
                x, label = x.to(device), label.to(device)
                output = model(x)
                loss = criterion(output, label)
                
                test_loss += loss.item()
                correct += torch.sum(output.argmax(dim=1) == label).item()
                total += len(label)
            test_losses.append(test_loss / len(test_loader))
            test_accs.append(correct / total)
        test_end = time.time()
        test_times.append(test_end-test_start)
        print("test_loss: {:.4f}, test_acc: {:.2f}%".format(test_losses[-1], test_accs[-1]*100))

    return train_losses, train_accs, train_times, test_losses, test_accs, test_times

## 2 Conv1D layers, activation maps [batch_size, 512], 1 linear layer [512, 5]

In [8]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=16, 
                               kernel_size=7, 
                               padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(in_channels=16, 
                               out_channels=16, 
                               kernel_size=5, 
                               padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 16
        
        self.linear = nn.Linear(32 * 16, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 32 * 16)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [9]:
for i in range(run):
    ecgnet = ECGConv1D()
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(i, ecgnet.to(device))

Epoch 1 - loss: 1.3805, acc: 54.32% / test_loss: 1.2487, test_acc: 68.16%
Epoch 2 - loss: 1.2284, acc: 68.98% / test_loss: 1.1855, test_acc: 73.78%
Epoch 3 - loss: 1.1741, acc: 74.65% / test_loss: 1.1534, test_acc: 75.73%
Epoch 4 - loss: 1.1537, acc: 76.06% / test_loss: 1.1391, test_acc: 77.12%
Epoch 5 - loss: 1.1372, acc: 77.52% / test_loss: 1.1155, test_acc: 79.53%
Epoch 6 - loss: 1.1179, acc: 79.34% / test_loss: 1.1043, test_acc: 80.60%
Epoch 7 - loss: 1.1117, acc: 79.77% / test_loss: 1.0998, test_acc: 81.02%
Epoch 8 - loss: 1.1083, acc: 79.95% / test_loss: 1.0959, test_acc: 81.30%
Epoch 9 - loss: 1.1054, acc: 80.20% / test_loss: 1.0947, test_acc: 81.37%
Epoch 10 - loss: 1.1020, acc: 80.50% / test_loss: 1.0923, test_acc: 81.46%
Epoch 11 - loss: 1.0996, acc: 80.66% / test_loss: 1.0900, test_acc: 81.74%
Epoch 12 - loss: 1.0974, acc: 80.90% / test_loss: 1.0879, test_acc: 81.83%
Epoch 13 - loss: 1.0953, acc: 81.00% / test_loss: 1.0868, test_acc: 81.84%
Epoch 14 - loss: 1.0940, acc: 81.0

In [17]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 90.28 at epoch 353
avg training time for one epoch: 0.78s
avg testing time for one epoch: 0.29s


## 2 Conv1D layers, activation maps: `[batch_size, 256]` (`stride = 2` for the first conv layer), 1 linear layer `[256, 5]` 

In [51]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=16, 
                               kernel_size=7, 
                               padding=3,
                               stride=2)  # 64 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 32 x 16
        self.conv2 = nn.Conv1d(in_channels=16, 
                               out_channels=16, 
                               kernel_size=5, 
                               padding=2)  # 32 x 16
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 16 x 16 = 256
        
        self.linear = nn.Linear(16 * 16, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 16 * 16)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [52]:
for i in range(run):
    ecgnet = ECGConv1D()
    torch.save(ecgnet.state_dict(), './weights/init_weight_256.pth')  # save trained weights    
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(i, ecgnet.to(device))
    torch.save(ecgnet.state_dict(), './weights/trained_weight_256.pth')  # save trained weights    

Epoch 1 - loss: 0.0049, acc: 15.00% / test_loss: 1.6064, test_acc: 22.69%
Epoch 2 - loss: 0.0048, acc: 22.50% / test_loss: 1.6013, test_acc: 23.59%
Epoch 3 - loss: 0.0048, acc: 22.50% / test_loss: 1.5961, test_acc: 24.27%
Epoch 4 - loss: 0.0048, acc: 35.00% / test_loss: 1.5918, test_acc: 27.29%
Epoch 5 - loss: 0.0048, acc: 25.00% / test_loss: 1.5894, test_acc: 22.65%
Epoch 6 - loss: 0.0047, acc: 27.50% / test_loss: 1.5880, test_acc: 22.65%
Epoch 7 - loss: 0.0047, acc: 27.50% / test_loss: 1.5862, test_acc: 22.65%
Epoch 8 - loss: 0.0047, acc: 27.50% / test_loss: 1.5837, test_acc: 22.65%
Epoch 9 - loss: 0.0047, acc: 27.50% / test_loss: 1.5804, test_acc: 23.87%
Epoch 10 - loss: 0.0047, acc: 25.00% / test_loss: 1.5762, test_acc: 25.21%
Epoch 11 - loss: 0.0047, acc: 30.00% / test_loss: 1.5708, test_acc: 32.00%
Epoch 12 - loss: 0.0046, acc: 42.50% / test_loss: 1.5636, test_acc: 39.87%
Epoch 13 - loss: 0.0046, acc: 47.50% / test_loss: 1.5544, test_acc: 40.37%
Epoch 14 - loss: 0.0046, acc: 55.0

KeyboardInterrupt: 

In [40]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 90.48 at epoch 338
avg training time for one epoch: 0.69s
avg testing time for one epoch: 0.30s


## 2 Conv1D layers, activation maps: `[batch_size, 256]` (`out_channels=8` for the second Conv1D layer), 1 linear layer `[256, 5]`

In [25]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=16, 
                               kernel_size=7, 
                               padding=3,
                               stride=1)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=5, 
                               padding=2)  # 64 x 8
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 8 = 256
        
        self.linear = nn.Linear(256, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 256)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [26]:
for i in range(run):
    ecgnet = ECGConv1D()
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(i, ecgnet.to(device))
    

Epoch 1 - loss: 1.4269, acc: 50.74% / test_loss: 1.2730, test_acc: 68.19%
Epoch 2 - loss: 1.2507, acc: 67.50% / test_loss: 1.2286, test_acc: 69.40%
Epoch 3 - loss: 1.2226, acc: 69.05% / test_loss: 1.2001, test_acc: 71.51%
Epoch 4 - loss: 1.1936, acc: 71.91% / test_loss: 1.1677, test_acc: 74.31%
Epoch 5 - loss: 1.1680, acc: 74.52% / test_loss: 1.1594, test_acc: 74.87%
Epoch 6 - loss: 1.1494, acc: 76.47% / test_loss: 1.1316, test_acc: 78.07%
Epoch 7 - loss: 1.1296, acc: 78.24% / test_loss: 1.1222, test_acc: 78.61%
Epoch 8 - loss: 1.1198, acc: 78.98% / test_loss: 1.1105, test_acc: 79.60%
Epoch 9 - loss: 1.1130, acc: 79.48% / test_loss: 1.1030, test_acc: 80.51%
Epoch 10 - loss: 1.1093, acc: 79.78% / test_loss: 1.0994, test_acc: 80.69%
Epoch 11 - loss: 1.1066, acc: 79.95% / test_loss: 1.0970, test_acc: 80.91%
Epoch 12 - loss: 1.1045, acc: 80.14% / test_loss: 1.0948, test_acc: 81.06%
Epoch 13 - loss: 1.1023, acc: 80.28% / test_loss: 1.0927, test_acc: 81.20%
Epoch 14 - loss: 1.1006, acc: 80.5

In [27]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 90.37 at epoch 374
avg training time for one epoch: 0.79s
avg testing time for one epoch: 0.30s


## 2 Conv1D layers, activation maps: `[batch_size, 128]` (`stride=2` for the first Conv1D layer, `out_channels=8` for the second Conv1D layer), 1 linear layer `[128, 5]`

In [19]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=16, 
                               kernel_size=7, 
                               padding=3,
                               stride=2)  # 64 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 32 x 16
        self.conv2 = nn.Conv1d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=5, 
                               padding=2)  # 32 x 8
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 16 x 8 = 128
        
        self.linear = nn.Linear(128, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 128)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [20]:
for i in range(run):
    ecgnet = ECGConv1D()
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(1, ecgnet.to(device))

Epoch 1 - loss: 1.4813, acc: 43.82% / test_loss: 1.3460, test_acc: 61.44%
Epoch 2 - loss: 1.3059, acc: 62.48% / test_loss: 1.2768, test_acc: 65.33%
Epoch 3 - loss: 1.2686, acc: 64.95% / test_loss: 1.2514, test_acc: 67.41%
Epoch 4 - loss: 1.2498, acc: 66.75% / test_loss: 1.2358, test_acc: 68.50%
Epoch 5 - loss: 1.2360, acc: 67.91% / test_loss: 1.2227, test_acc: 69.24%
Epoch 6 - loss: 1.2250, acc: 68.89% / test_loss: 1.2129, test_acc: 69.96%
Epoch 7 - loss: 1.2156, acc: 69.49% / test_loss: 1.2020, test_acc: 70.96%
Epoch 8 - loss: 1.2064, acc: 70.40% / test_loss: 1.1950, test_acc: 71.36%
Epoch 9 - loss: 1.1962, acc: 71.23% / test_loss: 1.1742, test_acc: 74.17%
Epoch 10 - loss: 1.1590, acc: 76.29% / test_loss: 1.1338, test_acc: 79.21%
Epoch 11 - loss: 1.1316, acc: 78.85% / test_loss: 1.1186, test_acc: 80.26%
Epoch 12 - loss: 1.1216, acc: 79.36% / test_loss: 1.1102, test_acc: 80.57%
Epoch 13 - loss: 1.1157, acc: 79.61% / test_loss: 1.1048, test_acc: 80.88%
Epoch 14 - loss: 1.1106, acc: 79.9

In [21]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 88.98 at epoch 386
avg training time for one epoch: 0.65s
avg testing time for one epoch: 0.28s


## 2 Conv1D layers, activation maps: `[batch_size, 64]` (`stride=2` for both Conv1D layers, `out_channels=8` for the second Conv1D layer), 1 linear layer `[64, 5]`

In [41]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=16, 
                               kernel_size=7, 
                               padding=3,
                               stride=2)  # 64 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 32 x 16
        self.conv2 = nn.Conv1d(in_channels=16, 
                               out_channels=8, 
                               kernel_size=5, 
                               padding=2,
                               stride=2)  # 16 x 8
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 8 x 8 = 64
        
        self.linear = nn.Linear(64, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 64)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [42]:
for i in range(run):
    ecgnet = ECGConv1D()
    torch.save(ecgnet.state_dict(), './weights/init_weight_64.pth')  # save trained weights    
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(i, ecgnet.to(device))
    torch.save(ecgnet.state_dict(), './weights/trained_weight_64.pth')  # save trained weights    

Epoch 1 - loss: 1.5153, acc: 39.37% / test_loss: 1.4064, test_acc: 54.38%
Epoch 2 - loss: 1.3197, acc: 63.06% / test_loss: 1.2732, test_acc: 67.88%
Epoch 3 - loss: 1.2540, acc: 67.96% / test_loss: 1.2359, test_acc: 69.82%
Epoch 4 - loss: 1.2269, acc: 69.63% / test_loss: 1.2158, test_acc: 71.03%
Epoch 5 - loss: 1.2112, acc: 70.71% / test_loss: 1.2014, test_acc: 71.75%
Epoch 6 - loss: 1.2001, acc: 71.41% / test_loss: 1.1907, test_acc: 72.42%
Epoch 7 - loss: 1.1926, acc: 71.99% / test_loss: 1.1846, test_acc: 72.71%
Epoch 8 - loss: 1.1862, acc: 72.34% / test_loss: 1.1757, test_acc: 73.61%
Epoch 9 - loss: 1.1745, acc: 73.47% / test_loss: 1.1564, test_acc: 75.51%
Epoch 10 - loss: 1.1537, acc: 75.83% / test_loss: 1.1363, test_acc: 77.75%
Epoch 11 - loss: 1.1361, acc: 77.86% / test_loss: 1.1226, test_acc: 79.06%
Epoch 12 - loss: 1.1267, acc: 78.66% / test_loss: 1.1145, test_acc: 79.86%
Epoch 13 - loss: 1.1205, acc: 79.15% / test_loss: 1.1099, test_acc: 80.19%
Epoch 14 - loss: 1.1161, acc: 79.4

In [43]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 88.96 at epoch 384
avg training time for one epoch: 0.68s
avg testing time for one epoch: 0.29s


## 2 Conv1D layers, activation maps: `[batch_size, 32]` (`stride=2` for both Conv1D layers, `out_channels=8` for the first Conv1D layer, `out_channels=4` for the second Conv1D layer), 1 linear layer `[32, 5]`

In [15]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=8, 
                               kernel_size=7, 
                               padding=3,
                               stride=2)  # 64 x 8
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 32 x 8
        self.conv2 = nn.Conv1d(in_channels=8, 
                               out_channels=4, 
                               kernel_size=5, 
                               padding=2,
                               stride=2)  # 16 x 4
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 8 x 4 = 32
        
        self.linear = nn.Linear(32, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 32)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [37]:
for i in range(run):
    ecgnet = ECGConv1D()
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(i, ecgnet.to(device))

Epoch 1 - loss: 1.5444, acc: 33.99% / test_loss: 1.4821, test_acc: 46.86%
Epoch 2 - loss: 1.4454, acc: 49.68% / test_loss: 1.3924, test_acc: 54.99%
Epoch 3 - loss: 1.3482, acc: 60.20% / test_loss: 1.3097, test_acc: 64.51%
Epoch 4 - loss: 1.2894, acc: 64.98% / test_loss: 1.2670, test_acc: 67.20%
Epoch 5 - loss: 1.2613, acc: 66.41% / test_loss: 1.2451, test_acc: 68.34%
Epoch 6 - loss: 1.2468, acc: 67.41% / test_loss: 1.2324, test_acc: 68.91%
Epoch 7 - loss: 1.2373, acc: 68.04% / test_loss: 1.2236, test_acc: 69.51%
Epoch 8 - loss: 1.2301, acc: 68.58% / test_loss: 1.2168, test_acc: 69.93%
Epoch 9 - loss: 1.2244, acc: 68.89% / test_loss: 1.2113, test_acc: 70.22%
Epoch 10 - loss: 1.2197, acc: 69.17% / test_loss: 1.2067, test_acc: 70.59%
Epoch 11 - loss: 1.2156, acc: 69.41% / test_loss: 1.2030, test_acc: 71.05%
Epoch 12 - loss: 1.2115, acc: 69.80% / test_loss: 1.1990, test_acc: 71.67%
Epoch 13 - loss: 1.2058, acc: 70.42% / test_loss: 1.1944, test_acc: 72.11%
Epoch 14 - loss: 1.1983, acc: 71.2

In [38]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 83.65 at epoch 375
avg training time for one epoch: 0.68s
avg testing time for one epoch: 0.30s


## 2 Conv1D layers, activation maps: `[batch_size, 16]` (`stride=4` for the first Conv1D, `stride=2` for the second Conv1D layer, `out_channels=8` for the first Conv1D layer, `out_channels=4` for the second Conv1D layer), 1 linear layer `[16, 5]`

In [35]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=8, 
                               kernel_size=7,
                               padding=3,
                               stride=4)  # 32 x 8
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 16 x 8
        self.conv2 = nn.Conv1d(in_channels=8, 
                               out_channels=4, 
                               kernel_size=5, 
                               padding=2,
                               stride=2)  # 8 x 4
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 4 x 4 = 16
        
        self.linear = nn.Linear(16, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 16)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [36]:
for i in range(run):
    ecgnet = ECGConv1D()
    torch.save(ecgnet.state_dict(), 'init_weight_16.pth')  # save init weights
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(1, ecgnet.to(device))
    torch.save(ecgnet.state_dict(), 'trained_weight_16.pth')  # save trained weights    

Epoch 1 - loss: 1.5913, acc: 22.90% / test_loss: 1.5805, test_acc: 25.07%
Epoch 2 - loss: 1.4801, acc: 43.91% / test_loss: 1.3970, test_acc: 53.35%
Epoch 3 - loss: 1.3732, acc: 55.30% / test_loss: 1.3480, test_acc: 57.83%
Epoch 4 - loss: 1.3330, acc: 60.41% / test_loss: 1.3133, test_acc: 62.90%
Epoch 5 - loss: 1.3040, acc: 63.41% / test_loss: 1.2879, test_acc: 65.62%
Epoch 6 - loss: 1.2828, acc: 65.27% / test_loss: 1.2693, test_acc: 66.81%
Epoch 7 - loss: 1.2666, acc: 66.58% / test_loss: 1.2544, test_acc: 68.22%
Epoch 8 - loss: 1.2537, acc: 67.74% / test_loss: 1.2425, test_acc: 69.03%
Epoch 9 - loss: 1.2431, acc: 68.53% / test_loss: 1.2331, test_acc: 69.74%
Epoch 10 - loss: 1.2346, acc: 69.08% / test_loss: 1.2256, test_acc: 70.06%
Epoch 11 - loss: 1.2279, acc: 69.49% / test_loss: 1.2196, test_acc: 70.36%
Epoch 12 - loss: 1.2226, acc: 69.84% / test_loss: 1.2147, test_acc: 70.63%
Epoch 13 - loss: 1.2183, acc: 70.06% / test_loss: 1.2105, test_acc: 70.75%
Epoch 14 - loss: 1.2145, acc: 70.3

In [27]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 84.24 at epoch 396
avg training time for one epoch: 0.67s
avg testing time for one epoch: 0.29s


## 2 Conv1D layers, activation maps: `[batch_size, 8]` (`stride=4` for both Conv1D layers, `out_channels=8` for the first Conv1D layer, `out_channels=4` for the second Conv1D layer), 1 linear layer `[8, 5]`

In [44]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=8, 
                               kernel_size=7,
                               padding=3,
                               stride=4)  # 32 x 8
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 16 x 8
        self.conv2 = nn.Conv1d(in_channels=8, 
                               out_channels=4, 
                               kernel_size=5, 
                               padding=2,
                               stride=4)  # 4 x 4
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 2 x 4 = 8
        
        self.linear = nn.Linear(8, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 8)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [45]:
ecgnet = ECGConv1D()
torch.save(ecgnet.state_dict(), 'init_weight_8.pth')  # save init weights
train_losses, train_accs, train_times, \
    test_losses, test_accs, test_times = train(i, ecgnet.to(device))

Epoch 1 - loss: 1.5771, acc: 28.76% / test_loss: 1.5401, test_acc: 40.01%
Epoch 2 - loss: 1.4975, acc: 42.33% / test_loss: 1.4613, test_acc: 45.10%
Epoch 3 - loss: 1.4493, acc: 45.92% / test_loss: 1.4298, test_acc: 47.15%
Epoch 4 - loss: 1.4163, acc: 49.27% / test_loss: 1.3920, test_acc: 53.10%
Epoch 5 - loss: 1.3695, acc: 57.26% / test_loss: 1.3445, test_acc: 59.01%
Epoch 6 - loss: 1.3263, acc: 61.52% / test_loss: 1.3109, test_acc: 62.50%
Epoch 7 - loss: 1.2992, acc: 63.52% / test_loss: 1.2893, test_acc: 64.11%
Epoch 8 - loss: 1.2824, acc: 64.36% / test_loss: 1.2749, test_acc: 65.04%
Epoch 9 - loss: 1.2710, acc: 64.98% / test_loss: 1.2648, test_acc: 65.65%
Epoch 10 - loss: 1.2628, acc: 65.59% / test_loss: 1.2570, test_acc: 66.12%
Epoch 11 - loss: 1.2566, acc: 65.90% / test_loss: 1.2510, test_acc: 66.45%
Epoch 12 - loss: 1.2516, acc: 66.21% / test_loss: 1.2463, test_acc: 66.63%
Epoch 13 - loss: 1.2474, acc: 66.46% / test_loss: 1.2424, test_acc: 66.77%
Epoch 14 - loss: 1.2438, acc: 66.6

In [46]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 82.57 at epoch 379
avg training time for one epoch: 0.66s
avg testing time for one epoch: 0.29s


## 2 Conv1D layers, activation maps: `[batch_size, 4]` (`stride=4` for both Conv1D layers, `out_channels=8` for the first Conv1D layer, `out_channels=2` for the second Conv1D layer), 1 linear layer `[4, 5]`

In [50]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=8, 
                               kernel_size=7,
                               padding=3,
                               stride=4)  # 32 x 8
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 16 x 8
        self.conv2 = nn.Conv1d(in_channels=8, 
                               out_channels=2, 
                               kernel_size=5, 
                               padding=2,
                               stride=4)  # 4 x 2
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 2 x 2 = 4
        
        self.linear = nn.Linear(4, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 4)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [51]:
for i in range(run):
    ecgnet = ECGConv1D()
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(i, ecgnet.to(device))

Epoch 1 - loss: 1.5790, acc: 27.07% / test_loss: 1.5475, test_acc: 39.20%
Epoch 2 - loss: 1.5187, acc: 39.48% / test_loss: 1.4927, test_acc: 42.00%
Epoch 3 - loss: 1.4823, acc: 44.85% / test_loss: 1.4662, test_acc: 46.93%
Epoch 4 - loss: 1.4601, acc: 47.32% / test_loss: 1.4455, test_acc: 48.35%
Epoch 5 - loss: 1.4395, acc: 49.21% / test_loss: 1.4256, test_acc: 50.53%
Epoch 6 - loss: 1.4197, acc: 51.33% / test_loss: 1.4076, test_acc: 52.68%
Epoch 7 - loss: 1.4019, acc: 52.63% / test_loss: 1.3920, test_acc: 53.93%
Epoch 8 - loss: 1.3854, acc: 54.50% / test_loss: 1.3754, test_acc: 55.86%
Epoch 9 - loss: 1.3632, acc: 57.21% / test_loss: 1.3512, test_acc: 58.46%
Epoch 10 - loss: 1.3424, acc: 59.03% / test_loss: 1.3370, test_acc: 59.40%
Epoch 11 - loss: 1.3309, acc: 59.67% / test_loss: 1.3278, test_acc: 59.77%
Epoch 12 - loss: 1.3230, acc: 60.00% / test_loss: 1.3212, test_acc: 60.08%
Epoch 13 - loss: 1.3172, acc: 60.32% / test_loss: 1.3161, test_acc: 60.26%
Epoch 14 - loss: 1.3127, acc: 60.4

In [52]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 74.96 at epoch 396
avg training time for one epoch: 0.69s
avg testing time for one epoch: 0.30s


## 2 Conv1D layers, activation maps: `[batch_size, 2]` (`stride=4` for both Conv1D layers, `out_channels=8` for the first Conv1D layer, `out_channels=1` for the second Conv1D layer), 1 linear layer `[2, 5]`

In [47]:
class ECGConv1D(nn.Module):
    def __init__(self):
        super(ECGConv1D, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, 
                               out_channels=8, 
                               kernel_size=7,
                               padding=3,
                               stride=4)  # 32 x 8
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 16 x 8
        self.conv2 = nn.Conv1d(in_channels=8, 
                               out_channels=1, 
                               kernel_size=5, 
                               padding=2,
                               stride=4)  # 4 x 1
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 2 x 1 = 2
        
        self.linear = nn.Linear(2, 5)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 2)
        x = self.linear(x)
        x = self.softmax(x)
        return x

In [48]:
for i in range(run):
    ecgnet = ECGConv1D()
    train_losses, train_accs, train_times, \
        test_losses, test_accs, test_times = train(i, ecgnet.to(device))

Epoch 1 - loss: 1.5941, acc: 23.91% / test_loss: 1.5856, test_acc: 31.51%
Epoch 2 - loss: 1.5759, acc: 29.58% / test_loss: 1.5639, test_acc: 32.10%
Epoch 3 - loss: 1.5551, acc: 31.21% / test_loss: 1.5438, test_acc: 34.58%
Epoch 4 - loss: 1.5381, acc: 33.98% / test_loss: 1.5270, test_acc: 37.56%
Epoch 5 - loss: 1.5224, acc: 37.63% / test_loss: 1.5101, test_acc: 41.03%
Epoch 6 - loss: 1.5059, acc: 40.66% / test_loss: 1.4926, test_acc: 41.99%
Epoch 7 - loss: 1.4886, acc: 42.05% / test_loss: 1.4745, test_acc: 43.25%
Epoch 8 - loss: 1.4717, acc: 43.68% / test_loss: 1.4599, test_acc: 44.33%
Epoch 9 - loss: 1.4585, acc: 44.98% / test_loss: 1.4477, test_acc: 45.91%
Epoch 10 - loss: 1.4472, acc: 46.26% / test_loss: 1.4369, test_acc: 46.84%
Epoch 11 - loss: 1.4369, acc: 47.18% / test_loss: 1.4266, test_acc: 47.80%
Epoch 12 - loss: 1.4268, acc: 47.87% / test_loss: 1.4162, test_acc: 49.60%
Epoch 13 - loss: 1.4162, acc: 49.48% / test_loss: 1.4052, test_acc: 53.43%
Epoch 14 - loss: 1.4048, acc: 52.5

In [49]:
print(f'best test accuracy: {max(test_accs)*100:.2f} at epoch {np.array(test_accs).argmax() + 1}')
print(f'avg training time for one epoch: {np.mean(train_times):.2f}s') 
print(f'avg testing time for one epoch: {np.mean(test_times):.2f}s')

best test accuracy: 70.59 at epoch 399
avg training time for one epoch: 0.66s
avg testing time for one epoch: 0.29s
